# CIECAM02 color appearance model
This program is for calculate parameters from CIE XYZ to CIECAM02 color apperance model.

Reference:
- https://en.wikipedia.org/wiki/CIECAM02
- https://en.wikipedia.org/wiki/LMS_color_space
- http://blog.sina.com.cn/s/blog_4b892b790100i5la.html

## Model description and background parameters
Background parameter.
Three surroundings in CIECAM02 model: average, dim, and dark.

| Surround condition | Surround ratio | $F$ | $c$   | $N_c$ | Application                     |
| ------------------ | -------------- | --- | ----- | ----- | ------------------------------- | 
| Average            | $S_R > 0.2$    | 1.0 | 0.69  | 1.0   | Viewing surface colors          |
| Dim                | $0<S_R<0.2$    | 0.9 | 0.59  | 0.95  | Viewing television              |
| Dark               | $S_R = 0$      | 0.8 | 0.525 | 0.8   | Using a projector in a dark room|

- $S_R$ = $L_{sw} / L_{dw}$: ratio of the absolute luminance of the reference white (white point) measured in the surround field to the display area. The 0.2 coefficient derives from the "gray world" assumption (~18%–20% reflectivity). It tests whether the surround luminance is darker or brighter than medium gray.
- $F$: factor determining degree of adaptation.
- $c$: impact of surrounding.
- $N_c$: chromatic induction factor.

Adoped white in test illuminant: 
$X_w$, $Y_w$, $Z_w$ and $Y_w = 100$.

Background in test condition:
$Y_b$.

Reference white in reference illuminant:
$X_{wr} = Y_{wr} = Z_{wr} = 100$.

Luminance of test adapting field ($cd/m^2$):
$L_A$.

Hue circle:

| Index | Red   | Yellow | Green  | Blue   | Red    |
| ----- | ----- | ------ | ------ | ------ | ------ |
| i     | 1     | 2      | 3      | 4      | 5      |
|hi     | 20.14 | 90.00  | 164.25 | 237.53 | 380.14 |
|ei     | 0.8   | 0.7    | 1.0    | 1.2    | 0.8    |
|Hi     | 0.0   | 100.0  | 200.0  | 300.0  | 400.0  |

In [38]:
function XYZ2CIECAM02(X,Y,Z,X_w,Y_w,Z_w,Y_b,L_A,surround_cond)
#     The surrounding conditon parameter:
    if surround_cond == "average"
        F = 1; c = 0.69; N_c = 1.0;
    elseif surround_cond == "dim"
        F = 0.9; c = 0.59; N_c = 0.95;
    elseif surround_cond == "dark"
        F = 0.8; c = 0.525; N_c = 0.8;
    end
    
#     Color matrix:
    M_cat02 = [0.7328 0.4296 -0.1624; -0.7036 1.6975 0.0061; 0.0030 0.0136 0.9834];
#     M_cat02_inv = [1.0961 -0.2789 0.1827; 0.4544 0.4735 0.0721; -0.0096 -0.0057 1.0153];
    M_cat02_inv = M_cat02^(-1);
    M_hpe = [0.38971 0.68898 -0.07868; -0.22981 1.1834 0.04641; 0 0 1];
    
    
#     Step 1:
    RGB_w = M_cat02 * [X_w; Y_w; Z_w];
    RGB = M_cat02 * [X; Y; Z];
    
#     Step 2:
    RGB_w = M_cat02 * [X_w; Y_w; Z_w];
    D_RGB = Y_w*D./RGB_w + 1 - D;
    
#     Step 3:
    RGB_c = D_RGB .* RGB;
    RGB_w = M_cat02 * [X_w; Y_w; Z_w];
    
#     Step 4:
    k = 1/(5*L_A + 1);
    n = Y_b/Y_w;
    z = 1.48 + sqrt(n);
    N_bb = N_cb = 0.725 * (1/n)^0.2;
    F_L = 0.2*k^4*(5*L_A) + 0.1*(1-k^4)^2*(5*L_A)^(1/3);
    
#     Step 5:
    RGB_apo = M_hpe * M_cat02_inv * RGB_c;
    RGB_w_apo = M_hpe * M_cat02_inv * RGB_wc;
    
#     Step 6:
    RGB_a_apo = [step6(RGB_apo[1],F_L), step6(RGB_apo[2],F_L), step6(RGB_apo[3],F_L)];
    RGB_aw_apo = [step6(RGB_w_apo[1],F_L), step6(RGB_w_apo[2],F_L), step6(RGB_w_apo[3],F_L)];
    
#     Step 7:
    C_1 = RGB_a_apo[1] - RGB_a_apo[2];
    C_2 = RGB_a_apo[2] - RGB_a_apo[3];
    C_3 = RGB_a_apo[3] - RGB_a_apo[1];
    a = C_1 - (1/11)*C_2;
    b = 1/2*(C_2 - C_3)/4.5;
    h = cal_h(a,b);
    if h < 20.14
        h_apo = h + 360;
    else
        h_apo = h;
    end
    ei, Hi, hi, hi1, ei1 = cal_hue(h_apo);
    H = Hi + (100*(h_apo-hi)/ei) / ((h_apo-hi)/ei + (hi1-h_apo)/ei1);
    # Verison 1: the textbook
    et = ei + (ei1-ei)*(h_apo-hi)/(hi1-hi);
    # Verison 2: the blog and wiki page
#     et = 1/4*(cos(pi/1.8*h+2)+3.8)
    A = (2*RGB_a_apo[1] + RGB_a_apo[2] + (1/20)*RGB_a_apo[3]) * N_bb;
    A_w = (2*RGB_aw_apo[1] + RGB_aw_apo[2] + (1/20)*RGB_aw_apo[3]) * N_bb;
    J = 100 * (A/A_w)^(c*z);
    Q = (4/c)*sqrt(1/100*J) * (A_w+4) * F_L^(1/4);
    t = (50000/13 * N_c*N_cb*et*sqrt(a^2+b^2)) / (RGB_a_apo[1] + RGB_a_apo[2] + 21/20*RGB_a_apo[3]);
    C = t^0.9 * sqrt(1/100*J) * (1.64-0.29^n)^0.73;
    M = C*F_L^(1/4);
    s = 100 * sqrt(M/Q);
    
    return J, C, H, h, M, s, Q
end

function step6(R, F_L)
    if R >= 0
        R_a = (400*(F_L*abs(R)/100)^0.42)  /  (27.13 + (F_L*abs(R)/100)^0.42) + 0.1;
    else
        R_a = -1 *( (400*(F_L*abs(R)/100)^0.42)  /  (27.13 + (F_L*abs(R)/100)^0.42) + 0.1);
    end
    return R_a
end;

function cal_h(a,b)
    h = atan2(b,a);
    if h < 0
        h = h + 2*pi;
    end
    h = h / pi * 180;
    return h
end;

function cal_hue(h)
    if 20.14 < h <= 90
        ei = 0.8; Hi = 0; hi = 20.14; hi1 = 90; ei1 = 0.7;
    elseif 90 < h <= 164.25
        ei = 0.7; Hi = 100; hi = 90; hi1 = 164.25; ei1 = 1;
    elseif 164.25 < h <= 237.53
        ei = 1; Hi = 200; hi = 164.25; hi1 = 237.53; ei1 = 1.2;
    elseif 237.53 < h <= 380.14
        ei = 1.2; Hi = 300; hi = 237.53; hi1 = 380.14; ei1 = 0.8;
    end
    return ei, Hi, hi, hi1, ei1
end;

### Test data

In [39]:
X = 57.06; Y = 43.06; Z = 31.96;
X_w = 95.05; Y_w = 100.00; Z_w = 108.88;
L_A = 31.83;
F = 1.0; c = 0.69; N_c = 1.0;
Y_b = 20;

In [40]:
# X = 3.53; Y = 6.56; Z = 2.14;
# X_w = 109.85; Y_w = 100; Z_w = 35.58;
# L_A = 318.31;

In [41]:
X = 19.01; Y = 20; Z = 21.78;
X_w = 109.85; Y_w = 100; Z_w = 35.58;
L_A = 31.83;

In [42]:
J, C, H, h, M, s, Q = XYZ2CIECAM02(X,Y,Z,X_w,Y_w,Z_w,Y_b,L_A,"average")

(42.987980513849735,50.95992316718763,305.46239113669674,248.90422175943695,43.72327262438705,59.26627256384901,124.47938466616225)

### Step 1: tested!

In [6]:
M_cat02 = [0.7328 0.4296 -0.1624; -0.7036 1.6975 0.0061; 0.0030 0.0136 0.9834];
display(M_cat02)

3×3 Array{Float64,2}:
  0.7328  0.4296  -0.1624
 -0.7036  1.6975   0.0061
  0.003   0.0136   0.9834

In [7]:
RGB_w = M_cat02 * [X_w; Y_w; Z_w]

3-element Array{Float64,1}:
 117.68  
  92.6766
  36.6789

In [8]:
RGB = M_cat02 * [X; Y; Z]

3-element Array{Float64,1}:
 18.9855
 20.7074
 21.7475

### Step 2: tested!

In [9]:
D = F * (1 - (1/3.6)*exp((-1*L_A-42)/92))

0.8754980431415154

In [10]:
D_RGB = Y_w*D./RGB_w + 1 - D

3-element Array{Float64,1}:
 0.868468
 1.06918 
 2.51143 

### Step 3: tested!

In [11]:
RGB_c = D_RGB .* RGB

3-element Array{Float64,1}:
 16.4883
 22.14  
 54.6172

In [12]:
RGB_wc = D_RGB .* RGB_w

3-element Array{Float64,1}:
 102.201 
  99.0882
  92.1164

### Step 4: tested!

In [13]:
k = 1/(5*L_A + 1)

n = Y_b/Y_w

z = 1.48 + sqrt(n)

N_bb = N_cb = 0.725 * (1/n)^0.2

F_L = 0.2*k^4*(5*L_A) + 0.1*(1-k^4)^2*(5*L_A)^(1/3)

0.5419205063751793

### Step 5: tested!

In [14]:
M_cat02_inv = [1.0961 -0.2789 0.1827; 0.4544 0.4735 0.0721; -0.0096 -0.0057 1.0153]

3×3 Array{Float64,2}:
  1.0961  -0.2789  0.1827
  0.4544   0.4735  0.0721
 -0.0096  -0.0057  1.0153

In [15]:
M_hpe = [0.38971 0.68898 -0.07868; -0.22981 1.1834 0.04641; 0 0 1]

3×3 Array{Float64,2}:
  0.38971  0.68898  -0.07868
 -0.22981  1.1834    0.04641
  0.0      0.0       1.0    

In [16]:
RGB_apo = M_hpe * M_cat02_inv * RGB_c

RGB_w_apo = M_hpe * M_cat02_inv * RGB_wc

3-element Array{Float64,1}:
 101.106 
  99.3483
  91.9798

### Step 6: 1 Problem!!!
There is one problem in step6!!!!!!

In [17]:
function step6(R, F_L)
    if R >= 0
        R_a = (400*(F_L*abs(R)/100)^0.42)  /  (27.13 + (F_L*abs(R)/100)^0.42) + 0.1;
    else
        R_a = -1 *( (400*(F_L*abs(R)/100)^0.42)  /  (27.13 + (F_L*abs(R)/100)^0.42) + 0.1);
    end
    return R_a
end;

In [18]:
RGB_a_apo = [step6(RGB_apo[1],F_L), step6(RGB_apo[2],F_L), step6(RGB_apo[3],F_L)]

RGB_aw_apo = [step6(RGB_w_apo[1],F_L), step6(RGB_w_apo[2],F_L), step6(RGB_w_apo[3],F_L)]

3-element Array{Float64,1}:
 11.233 
 11.1535
 10.8109

### Step 7:
This step mainly based on the wiki page.

In [19]:
C_1 = RGB_a_apo[1] - RGB_a_apo[2]
C_2 = RGB_a_apo[2] - RGB_a_apo[3]
C_3 = RGB_a_apo[3] - RGB_a_apo[1]

a = C_1 - (1/11)*C_2

b = 1/2*(C_2 - C_3)/4.5

-0.626338080038736

In [20]:
function cal_h(a,b)
    h = atan2(b,a);
    if h < 0
        h = h + 2*pi;
    end
    h = h / pi * 180;
    return h
end;

Calculate hue circle:

| Index | Red   | Yellow | Green  | Blue   | Red    |
| ----- | ----- | ------ | ------ | ------ | ------ |
| i     | 1     | 2      | 3      | 4      | 5      |
|hi     | 20.14 | 90.00  | 164.25 | 237.53 | 380.14 |
|ei     | 0.8   | 0.7    | 1.0    | 1.2    | 0.8    |
|Hi     | 0.0   | 100.0  | 200.0  | 300.0  | 400.0  |

In [21]:
h = cal_h(a,b)

if h < 20.14
    h_apo = h + 360;
else
    h_apo = h;
end

248.88436720923212

In [22]:
function cal_hue(h)
    if 20.14 < h <= 90
        ei = 0.8; Hi = 0; hi = 20.14; hi1 = 90; ei1 = 0.7;
    elseif 90 < h <= 164.25
        ei = 0.7; Hi = 100; hi = 90; hi1 = 164.25; ei1 = 1;
    elseif 164.25 < h <= 237.53
        ei = 1; Hi = 200; hi = 164.25; hi1 = 237.53; ei1 = 1.2;
    elseif 237.53 < h <= 380.14
        ei = 1.2; Hi = 300; hi = 237.53; hi1 = 380.14; ei1 = 0.8;
    end
    return ei, Hi, hi, hi1, ei1
end;

In [23]:
ei, Hi, hi, hi1, ei1 = cal_hue(h_apo)

H = Hi + (100*(h_apo-hi)/ei) / ((h_apo-hi)/ei + (hi1-h_apo)/ei1)

305.4525961738253

!!! Problem: 

In [24]:
# Verison 1: the textbook
et = ei + (ei1-ei)*(h_apo-hi)/(hi1-hi)

# Verison 2: the blog and wiki page
# et = 1/4*(cos(pi/1.8*h+2)+3.8)

A = (2*RGB_a_apo[1] + RGB_a_apo[2] + (1/20)*RGB_a_apo[3]) * N_bb

A_w = (2*RGB_aw_apo[1] + RGB_aw_apo[2] + (1/20)*RGB_aw_apo[3]) * N_bb

J = 100 * (A/A_w)^(c*z)

Q = (4/c)*sqrt(1/100*J) * (A_w+4) * F_L^(1/4)

t = (50000/13 * N_c*N_cb*et*sqrt(a^2+b^2)) / (RGB_a_apo[1] + RGB_a_apo[2] + 21/20*RGB_a_apo[3])

C = t^0.9 * sqrt(1/100*J) * (1.64-0.29^n)^0.73

M = C*F_L^(1/4)

s = 100 * sqrt(M/Q)

59.271348342118536